# Integrating Subtle Ads in Text using Wen 2.5 3B and RAG
This notebook implements a comprehensive workflow to generate detailed, context-aware responses to user queries, with the optional integration of subtle advertisements. It covers data loading and preprocessing, embedding generation and context retrieval using FAISS, text generation with a state-of-the-art language model, subtle advertisement injection, and evaluation of ad integration using a zero-shot classifier.


# Setup and Data Loading

In this section, we install necessary packages and import all the libraries required for the project. We then load our dataset from a JSON Lines file, where each line represents one JSON object. Finally, we convert the loaded data into a Hugging Face `Dataset` to facilitate further processing and iteration.

**Key Points:**
- **Package Installation:**  
  The `faiss-gpu` package is installed for fast similarity search on GPU.
- **Imports:**  
  Libraries such as `os`, `json`, `numpy`, `faiss`, `torch`, and more are imported for file operations, data manipulation, and machine learning functionalities.
- **Data Loading:**  
  We read the JSONL file line-by-line, ensuring we handle any parsing errors, and then create a structured dataset from the parsed data.


In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [2]:
!pip install faiss-gpu-cu12 
!pip install spacy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 40.5 MB/s eta 0:00:00:00:0100:01


In [3]:
import os
import json
import math
import re
import numpy as np
import faiss
import torch
import spacy
from tqdm import tqdm
from datasets import Dataset
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline 


# List files in the Kaggle input directory (optional, for debugging)
!ls /kaggle/input

data_file = "/kaggle/input/touche-task4-2025/touche-task4-2025-segments_new.jsonl"

data_list = []
with open(data_file, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        line = line.strip()
        if line:
            try:
                data_list.append(json.loads(line))
            except Exception as e:
                print(f"Error parsing line {i}: {e}")

print("Loaded", len(data_list), "examples.")

# Create a Hugging Face Dataset from the list
dataset = Dataset.from_list(data_list)
print("Dataset created:", dataset)

2025-05-24 18:26:29.224658: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748111189.418940      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748111189.479101      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


task4subtask01	touche-task4-2025
Loaded 3309 examples.
Dataset created: Dataset({
    features: ['query', 'candidates', 'advertisements'],
    num_rows: 3309
})


# Embedding Setup and Context Retrieval

In this section, we set up the embedding generation and define helper functions to retrieve context for a given query.

**Details:**
- **Sentence Transformer:**  
  A pre-trained model (`all-MiniLM-L6-v2`) is loaded to convert text into numerical embeddings. These embeddings capture semantic information from the text.
- **`embed_text` Function:**  
  This function takes a text input and returns its embedding as a numerical vector.
- **`build_query_context` Function:**  
  For each query, candidate segments are extracted and embedded. Then, using FAISS (a library for fast similarity search), we compute the similarity between the query and the segments to retrieve the top-k most relevant segments. The function returns a concatenated context string and the corresponding document IDs.

These steps ensure that our language model will have contextually relevant information when generating responses.


In [4]:
# Load the sentence transformer for embeddings
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(embed_model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model.to(device)

def embed_text(text):
    return embed_model.encode(text, convert_to_numpy=True)
 #Converts input text into a numerical embedding vector.

reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

def build_query_context(query_item, top_k=3, initial_k=10):
    # --- 0. unpack ---
    query_text = query_item["query"]["text"]
    cand_list = [
        {
            "segment": c["doc"].get("segment", "").strip(),
            "docid":   c["docid"],
            "es_score": c.get("score", 0.0),
            "edu_val":  c.get("edu_value", 0.0)
        }
        for c in query_item["candidates"]
        if c["doc"].get("segment", "").strip()
    ]
    if not cand_list:
        return "", []
    
    # --- 1. FAISS: embed & retrieve top initial_k ---
    seg_texts = [c["segment"] for c in cand_list]
    embeddings = np.vstack([embed_text(s) for s in seg_texts])
    q_emb = embed_text(query_text).reshape(1, -1)
    
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    k_search = min(initial_k, len(seg_texts))
    _, idxs = index.search(q_emb, k_search)
    idxs = idxs[0]
    
    initial = [cand_list[i] for i in idxs]
    
    # --- 2. Cross‑Encoder rerank on those --- 
    pairs = [[query_text, c["segment"]] for c in initial]
    rerank_scores = reranker.predict(pairs)
    
    # --- 3. Build arrays of raw metrics ---
    es_scores  = np.array([c["es_score"]  for c in initial])
    edu_vals   = np.array([c["edu_val"]   for c in initial])
    rerank_sc  = np.array(rerank_scores)
    
    # --- 4. Convert to rank positions (1 = best) ---
    def to_ranks(arr, descending=True):
        # argsort twice gives 1-based ranks
        order = np.argsort(-arr if descending else arr)
        ranks = np.empty_like(order)
        ranks[order] = np.arange(1, len(arr)+1)
        return ranks
    
    r_rank = to_ranks(rerank_sc, descending=True)
    e_rank = to_ranks(edu_vals,   descending=True)
    s_rank = to_ranks(es_scores,  descending=True)
    
    # --- 5. Compute averages and collect full info ---
    enriched = []
    w1, w2, w3 = 1.0, 1.4 , 1.5
    for c, rr, er, sr in zip(initial, r_rank, e_rank, s_rank):
        avg = (rr*w1 + er*w2 + sr*w3) / 3
        enriched.append({
            "docid":  c["docid"],
            "segment":c["segment"],
            "r_rank": rr,
            "e_rank": er,
            "s_rank": sr,
            "avg_rank": avg
        })
    
    # print out all 10 so you can inspect the 3‑tuple ranks + avg
    print("All candidate rankings (docid: [rerank, edu, ES], avg):")
    for item in enriched:
        print(f"  {item['docid']}: [{item['r_rank']}, {item['e_rank']}, {item['s_rank']}], avg={item['avg_rank']:.2f}")

    
    # --- 6. pick top_k by lowest avg_rank ---
    enriched.sort(key=lambda x: x["avg_rank"])
    top = enriched[:top_k]

    # print out all 10 so you can inspect the 3‑tuple ranks + avg
    print("The 3 candidate segments being returned (docid: [rerank, edu, ES], avg):")
    for item in top:
        print(f"  {item['docid']}: [{item['r_rank']}, {item['e_rank']}, {item['s_rank']}], avg={item['avg_rank']:.2f}")

    
    # build the returned context string + docid list
    context = "\n".join(item["segment"] for item in top)
    docids  = [item["docid"] for item in top]
    return context, docids

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

# Model Loading and Text Generation Setup

This section is dedicated to loading the language model and tokenizer needed for generating text responses. We are using the Qwen2.5-3B model from Hugging Face.

**Highlights:**
- **Loading the Model:**  
  The model and its corresponding tokenizer are loaded using Hugging Face's `AutoTokenizer` and `AutoModelForCausalLM`.
- **Pipeline Creation:**  
  A text generation pipeline is set up for generating coherent responses from the model. The configuration automatically handles device allocation (CPU/GPU) and model precision.

This setup is critical as it defines the foundation for generating both base responses and modified responses with advertisements.


In [5]:
# ------------------------------
# Ad classifier with threshold adjustment
# ------------------------------
def ad_classifier(text: str, threshold: float = 0.95) -> int:
    """
    Returns 1 if the probability of LABEL_1 (ad) > threshold, else 0.
    """
    # Truncate text to model's max length (512 tokens)
    result = ad_detector(text[:512])[0]  # List of dicts for each class
    
    # Extract scores for LABEL_0 and LABEL_1
    score_label_0 = next((x["score"] for x in result if x["label"] == "LABEL_0"), 0.0)
    score_label_1 = next((x["score"] for x in result if x["label"] == "LABEL_1"), 0.0)
    
    # Predict 1 (ad) only if LABEL_1 score exceeds the threshold
    return 1 if score_label_1 > threshold else 0

# ------------------------------
# Evaluation function for ads (Class 1)
# ------------------------------
def evaluate_ad_integration(responses, ground_truth_labels, threshold=0.95):
    predictions = [ad_classifier(text, threshold) for text in responses]
    
    # Calculate metrics for Class 1 (ads)
    tp = np.sum((np.array(predictions) == 1) & (np.array(ground_truth_labels) == 1))
    fp = np.sum((np.array(predictions) == 1) & (np.array(ground_truth_labels) == 0))
    fn = np.sum((np.array(predictions) == 0) & (np.array(ground_truth_labels) == 1))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 1.0  # Avoid division by zero
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    
    return precision, recall, predictions

In [6]:
# !git clone https://github.com/taqiKaAccount/miniLM-baseline.git /kaggle/working/miniLM-baseline

In [7]:
# !mv /kaggle/working/miniLM-baseline /kaggle/working/my-minilm

In [ ]:
# Load the Qwen2.5-7B model from Hugging Face.

model_name = "Qwen/Qwen2.5-7B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

# Create a text-generation pipeline without passing a device parameter
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

from huggingface_hub import login
from transformers import pipeline, AutoModelForSequenceClassification
import json
import numpy as np

#add your own model here for ad detection

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


Label mapping: {0: 'LABEL_0', 1: 'LABEL_1'}


# Text Cleaning and Sentence Completion Functions

Here, we define functions to post-process the model’s text outputs to ensure that they are clean and properly formatted.

**Functions:**
- **`clean_response`:**  
  Removes unwanted elements such as HTML tags, markdown formatting, and extraneous whitespace to yield a clean text output.
- **`complete_sentence`:**  
  Checks whether the generated text ends with a complete sentence (proper punctuation). If not, it prompts the model to append additional text until the sentence is complete.

These steps help maintain the readability and professionalism of the generated responses.


In [9]:
def complete_sentence(text):
    """Enhanced sentence completion to avoid cut-offs"""
    if not text.strip().endswith(('.', '!', '?')):
        # Simplified approach - find the last complete sentence
        last_period = max(text.rfind('.'), text.rfind('!'), text.rfind('?'))
        if last_period > len(text) * 0.8:  # If near the end, truncate to avoid cut-off
            return text[:last_period+1]
        
        # Otherwise complete with model
        completion_prompt = f"{text}\n\nComplete this sentence:"
        completion = generator(
            completion_prompt,
            max_new_tokens=30,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            truncation=True
        )[0]["generated_text"]
        
        # Extract just the completion part
        if "Complete this sentence:" in completion:
            ending = completion.split("Complete this sentence:", 1)[1].strip()
            # Only take up to the first sentence ending punctuation
            for punct in ['.', '!', '?']:
                if punct in ending:
                    ending = ending.split(punct, 1)[0] + punct
                    break
            return text + " " + ending
    
    return text

def clean_response(text):
    """More robust response cleaning"""
    # Remove empty lines, excess whitespace, and known artifacts
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'\s+', ' ', text)
    
    # Remove instruction markers
    markers = ["Instructions:", "Original:", "Rewritten:", "Modified version:", 
               "Please rewrite", "Original text:", "Task:", "Modified text:"]
    for marker in markers:
        if marker in text:
            # Only keep text after the last occurrence of the marker
            text = text.split(marker)[-1].strip()
    
    # Remove any incomplete phrases at the end
    incomplete_phrases = [
        "I hope this helps", "Let me know if", "Is there anything", 
        "Do you have any", "If you have any"
    ]
    for phrase in incomplete_phrases:
        if text.endswith(phrase):
            text = text[:-(len(phrase))].strip()
    
    return text

In [10]:
# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

def calculate_rouge_score(original_text, modified_text, nlp):
    """Calculate ROUGE-1 F1 score between original and modified texts"""
    def get_lemmas(text):
        doc = nlp(text)
        return [token.lemma_.lower() for token in doc if not (token.is_stop or token.is_punct)]
    
    lemmas1 = get_lemmas(original_text)
    lemmas2 = get_lemmas(modified_text)
    
    n_match = len([l for l in lemmas1 if l in lemmas2])
    n_reference = len(lemmas1)
    n_candidate = len(lemmas2)
    
    try:
        r = n_match / n_reference if n_reference > 0 else 0
        p = n_match / n_candidate if n_candidate > 0 else 0
        
        if p == 0 and r == 0:
            return 0.0
        return 2 * (p * r) / (p + r)
    except ZeroDivisionError:
        return 0.0



def measure_ad_naturalness(response, ad_entry):
    """
    Measures how naturally an advertisement is integrated into the response text.
    Returns a score between 0.0 and 1.0
    """
    score = 0.0
    response_lower = response.lower()
    ad_text = ad_entry["item"].strip().lower()
    qualities = ad_entry["qualities"].lower()

    # Check 1: Ad presence (base score: 0.2)
    if ad_text not in response_lower:
        print(f"❌ Ad text '{ad_text}' not found in response")
        return 0.0
    score += 0.2

    # Check 2: Position scoring (up to 0.3)
    start_pos = response_lower.find(ad_text)
    text_length = len(response)
    relative_position = start_pos / text_length
    
    print(f"Ad position: {relative_position:.2f} of text length")
    
    if 0.3 <= relative_position <= 0.7:
        score += 0.3  # Ideal middle position
        print("✅ Ad in ideal middle position")
    elif 0.2 <= relative_position <= 0.8:
        score += 0.2  # Acceptable position
        print("✓ Ad in acceptable position")
    else:
        score += 0.1  # Poor position (beginning or end)
        print("⚠️ Ad too close to beginning or end")

    # Check 3: Context coherence (up to 0.3)
    sentences = response.split('.')
    ad_sentence = ""
    for sentence in sentences:
        if ad_text in sentence.lower():
            ad_sentence = sentence
            break
    
    if ad_sentence:
        quality_words = qualities.split()
        quality_count = sum(1 for word in quality_words if word in ad_sentence.lower())
        coherence_score = min(0.3, quality_count * 0.1)
        score += coherence_score
        print(f"Context coherence: {coherence_score:.2f} ({quality_count} quality terms used)")

    # Check 4: Avoid obvious ad markers (up to 0.2)
    obvious_markers = ['advertisement', 'sponsor', 'buy now', 'available at', 'check out']
    marker_count = sum(1 for marker in obvious_markers if marker in response_lower)
    marker_score = 0.2 * (1 - min(1, marker_count/len(obvious_markers)))
    score += marker_score
    print(f"Ad subtlety: {marker_score:.2f} ({marker_count} obvious markers found)")

    return round(min(1.0, score), 2)

def similar(s1, s2):
    """Check if sentences are very similar"""
    s1_words = set(s1.lower().split())
    s2_words = set(s2.lower().split())
    overlap = len(s1_words.intersection(s2_words))
    total = len(s1_words.union(s2_words))
    return overlap / total > 0.7 if total > 0 else False

# General Response Generation Functions

In this section, we define functions that handle the overall response generation process. These functions use the model pipeline to generate responses based on constructed prompts.

**Core Functions:**
- **`generate_response`:**  
  A generic function that takes a prompt and generates text up to a specified token limit. It cleans the output and ensures it ends with complete punctuation.
- **`generate_non_ad_response`:**  
  Constructs a detailed prompt that includes both the query and its related context, then generates a coherent, advertisement-free response.
- **`inject_ad`:**  
  Modifies an existing response by inserting advertisement content. It supports two ad styles:
  - **Explicit:** Inserts a clearly labeled advertisement as a new paragraph.
  - **Subtle:** Integrates the advertisement naturally into the existing text.
  
These functions form the backbone of our text generation and advertisement integration workflow.


In [11]:
def generate_response(prompt, max_new_tokens=256, stop_token="Modified Response:"):
    output = generator(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        truncation=True
    )
    response_text = output[0]["generated_text"]
    
    # Extract only the generated part after the prompt
    if prompt in response_text:
        response_text = response_text[len(prompt):].strip()
        
    # Remove specific instructions and examples that might be leaking
    for marker in ["Original:", "Rewritten:", "Please rewrite", "I love going to the", "I'll try to make it sound natural"]:
        if marker in response_text:
            parts = response_text.split(marker, 1)
            response_text = parts[0].strip()
    
    cleaned = clean_response(response_text)
    finalized = complete_sentence(cleaned)
    return finalized


def generate_non_ad_response(query_text, context): # ----------- Generates ad-free responses.
    base_prompt = (
        f"Query: {query_text}\n\nContext:\n{context}\n\n"
        "Please generate a detailed and coherent response that answers the query in plain text with no formatting symbols. "
        "Ensure that your response ends naturally at a complete sentence without exceeding the token limit.\n\n"
        "Response:\n"
    )
    # Use "Response:" as stop token to trim any prompt instructions.

    createdResponse = generate_response(base_prompt, max_new_tokens=256, stop_token="Response:")

    prediction = ad_classifier(createdResponse, threshold=0.95)
    if prediction == 0: #if the non ad response is not detected to have ad-like structure return it
        return createdResponse
    reps = 0
    while prediction !=0 or reps < 10: #keep going until its not like an ad or 10th trial
        reps+=1
        createdResponse = generate_response(base_prompt, max_new_tokens=256, stop_token="Response:")
        prediction = ad_classifier(createdResponse, threshold=0.95)

    
    return createdResponse

def inject_ad(response, ad_entry, ad_style="subtle", max_attempts=7):
    """
    Improved ad injection that preserves text completeness and ensures natural integration
    """
    print(f"Injecting ad: {ad_entry['item']} ({ad_entry['qualities']})")
    
    # Clean the original response first
    clean_original = clean_response(response)
    
    # Split the text at a natural break point (after a complete sentence) in the middle
    sentences = [s.strip() for s in clean_original.split('.') if s.strip()]
    middle_idx = max(1, len(sentences) // 2)
    
    # Create parts for insertion
    first_part = '. '.join(sentences[:middle_idx]) + '.'
    second_part = ' ' + '. '.join(sentences[middle_idx:])
    
    best_response = None
    best_score = 0
    
    # Adjust integration approach based on style
    subtlety = 0.7 if ad_style == "subtle" else 0.4
    
    for attempt in range(max_attempts):
        try:
            # Create a natural transition sentence with the ad
            ad_text = ad_entry['item']
            qualities = ad_entry['qualities']
            
            # Use ad qualities for dynamic transitions
            ad_keywords = ', '.join(qualities.split(','))
            
            # Different integration approaches based on attempt
            if attempt == 0:
                # Approach 1: Generate a natural transition
                subtlety_prompt = "be very subtle and not obvious" if ad_style == "subtle" else "be clear about recommending the product"
                prompt = f"""
Write a single sentence that naturally connects these two paragraphs while mentioning {ad_text} with its qualities ({ad_keywords}):

Paragraph 1: {first_part}
Paragraph 2: {second_part}

The transition sentence should:
- Sound natural in the context
- Mention {ad_text} and its qualities like {ad_keywords}
- Be concise (20-30 words)
- {subtlety_prompt}

Transition sentence:"""
                
                output = generator(
                    prompt,
                    max_new_tokens=60,
                    temperature=0.7,
                    top_p=0.9,
                    do_sample=True
                )
                
                transition = output[0]["generated_text"].split("Transition sentence:")[-1].strip()
                transition = re.sub(r'[\*\"\'\`]', '', transition)  # Remove formatting characters
                
                # Assemble the full response
                modified_response = first_part + " " + transition + second_part
                
            elif attempt == 1:
                # Approach 2: Insert an example or comparison
                if ad_style == "subtle":
                    transition = f"For instance, {ad_text} offers {qualities}, providing an option that aligns with these needs."
                else:
                    transition = f"I recommend considering {ad_text}, which offers {qualities} and addresses these needs effectively."
                    
                modified_response = first_part + " " + transition + second_part
                
            else:
                # Approach 3: Direct insertion with context adaptation
                # Find a relevant sentence to modify in the first half
                for i in range(max(0, middle_idx-3), middle_idx):
                    if any(word in sentences[i].lower() for word in ['solution', 'option', 'product', 'service']):
                        sentences[i] = sentences[i] + f" {ad_text} is notable for its {qualities}."
                        break
                else:
                    # If no suitable sentence found, just insert after the middle
                    sentences.insert(middle_idx, f"{ad_text} provides {qualities} for those seeking a solution.")
                
                modified_response = '. '.join(sentences) + '.'
            
            # Clean any artifacts
            modified_response = clean_response(modified_response)
            
            # Calculate metrics
            natural_score = measure_ad_naturalness(modified_response, ad_entry)
            rouge_score = calculate_rouge_score(clean_original, modified_response, nlp)
            clf_output      = ad_detector(modified_response[:512])[0]
            p_non_ad        = next(x['score'] for x in clf_output if x['label']=="LABEL_0")
            evasion_score   = p_non_ad   # higher means it fooled the detector
            
            # Keep your scoring formula
            total_score = (natural_score * 0.4 + rouge_score * 0.2 + evasion_score * 0.4)
            
            print(f"Attempt {attempt + 1}:")
            print(f"  Naturalness score: {natural_score:.2f}")
            print(f"  Evasion score:    {evasion_score:.2f}")
            print(f"  ROUGE score: {rouge_score:.2f}")
            print(f"  Total score: {total_score:.2f}")
            
            # Keep track of best response
            if total_score > best_score:
                best_response = modified_response
                best_score = total_score
                
            # Accept if good enough
            if natural_score >= subtlety and rouge_score >= 0.7 and evasion_score>0.8:
                print("✅ Met quality thresholds")
                return modified_response
                
        except Exception as e:
            print(f"Error in ad injection attempt {attempt+1}: {e}")
    
    # Return best response found or fallback
    if best_response:
        print(f"⚠️ Using best response (Score: {best_score:.2f})")
        return best_response
    
    # Final fallback - ensure we return something with the ad
    fallback = clean_original.split('.')
    middle = len(fallback) // 2
    fallback.insert(middle, f" {ad_entry['item']} provides {ad_entry['qualities']}")
    return '. '.join(fallback)
    




# Random Seed Initialization

To ensure the reproducibility of our experiments, we set a random seed across Python, NumPy, and PyTorch. This means that every time the code is run, the random components (such as data shuffling or random text generation) will produce the same results.


In [12]:
import json
import random
import numpy as np
import torch

# Optional: Set a random seed for consistency
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

# Processing Queries and Generating Responses

In this section, we process each query from our dataset and generate responses using our previously defined functions.

**Workflow:**
- **Query Loop:**  
  Iterate through each query in the dataset (limited for testing purposes).
- **Context Retrieval:**  
  For each query, retrieve the top-3 relevant document segments and their document IDs using `build_query_context`.
- **Response Generation:**  
  For each query:
  - Generate a base response without an advertisement if the advertisement entry is `None`.
  - If an advertisement is provided, modify the base response to include it (using `inject_ad`).
- **Response Structuring:**  
  Each generated response is stored as a dictionary with keys such as query ID, topic, references, response text, and advertisement label.
- **Output and Saving:**  
  The generated responses are printed (for testing) and then saved to a JSONL file for later use.

This modular approach helps ensure clarity and maintainability of the code.


In [14]:
# ------------------------------
# Assume the following helper functions are already defined:
# - build_query_context(query_item, top_k=3)
# - generate_non_ad_response(query_text, context)
# - inject_ad(base_response, ad_entry, ad_style="subtle")
# - clean_response(text) is used inside those functions.
# ------------------------------

responses_output = []
reps = 0
# Loop over every query in the dataset
for query_item in dataset:
    if reps>60:
        break
    reps+=1
    
    query_id = query_item["query"]["id"]
    query_text = query_item["query"]["text"]

    # Retrieve context (top 3 candidate segments)
    context, retrieved_docids = build_query_context(query_item, top_k=3)

    # Initialize a variable to store the base (non-ad) response.
    base_response = None

    # Process the advertisement list.
    # According to the task, the advertisement list always contains one entry that is None.
    # For each advertisement entry:
    #   - If the entry is None: generate the non-ad response.
    #   - Otherwise, use the base response as input for ad injection.
    ad_list = query_item.get("advertisements", [])

    # We'll generate one response per advertisement entry.
    for idx, ad in enumerate(ad_list):
        # For the None entry, generate the base non-ad response ie same as input
        if ad is None:
            base_response = generate_non_ad_response(query_text, context)
            base_response_save = base_response
            response_text = base_response
            ad_label = ""  # No advertisement label for the base response.
        else:
            # Ensure the base response is available.
            if base_response is None:
                base_response = generate_non_ad_response(query_text, context)
                base_response_save = base_response
                
            
            # 1) Inject until we see the ad_text appear
            ad_text = ad["item"]
            max_trials = 5
            trial = 0
            injected = ""
            
            while trial < max_trials:
                injected = inject_ad(base_response, ad, ad_style="subtle")
                if ad_text.lower() in injected.lower():
                    # success!
                    break
                trial += 1
            
            # 2) Use whatever we got on success (or the last try if we never saw the ad)
            response_text = injected
            ad_label     = ad_text


         # structuting each response 
        responses_output.append({
            "id": f"{query_id}-{idx}",
            "topic": query_text,
            "references": retrieved_docids,
            "response": response_text,
            "advertisement": ad_label,
            "tag": "GitGud_RAG_Qwen2.5-7B"
        })

#==================================
# For TESTING purposes, print some sample outputs
#======================================
i = 0
for entry in responses_output:
    if i > 5 and i< 31: #stop after 12 outputs
        print(f"Response ID: {entry['id']}")
        print("Topic:", entry["topic"])
        print("Response:", entry["response"])
        print("Advertisement:", entry["advertisement"])
        print("References:", entry["references"])
        print("-" * 50) 
    
    if i>30:
        break

    i+=1

# ------------------------------
# Save the Responses to a JSONL File
# ------------------------------
output_file = "/kaggle/working/subtask1_responses.jsonl"
with open(output_file, "w", encoding="utf-8") as fout:
    for entry in responses_output:
        fout.write(json.dumps(entry) + "\n")
print(f"Generated {len(responses_output)} responses and saved to {output_file}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

All candidate rankings (docid: [rerank, edu, ES], avg):
  msmarco_v2.1_doc_06_440399025#3_774259821: [1, 8, 2], avg=5.07
  msmarco_v2.1_doc_56_928298929#13_1922015453: [9, 5, 9], avg=9.83
  msmarco_v2.1_doc_56_928298929#16_1922022003: [8, 10, 7], avg=10.83
  msmarco_v2.1_doc_56_928298929#8_1922004478: [4, 1, 4], avg=3.80
  msmarco_v2.1_doc_56_928298929#5_1921998165: [6, 7, 8], avg=9.27
  msmarco_v2.1_doc_56_928298929#12_1922013332: [7, 9, 6], avg=9.53
  msmarco_v2.1_doc_06_440399025#2_774258259: [2, 6, 1], avg=3.97
  msmarco_v2.1_doc_56_54247280#0_116503893: [10, 2, 10], avg=9.27
  msmarco_v2.1_doc_56_928298929#0_1921988401: [3, 4, 3], avg=4.37
  msmarco_v2.1_doc_56_928298929#1_1921990313: [5, 3, 5], avg=5.57
The 3 candidate segments being returned (docid: [rerank, edu, ES], avg):
  msmarco_v2.1_doc_56_928298929#8_1922004478: [4, 1, 4], avg=3.80
  msmarco_v2.1_doc_06_440399025#2_774258259: [2, 6, 1], avg=3.97
  msmarco_v2.1_doc_56_928298929#0_1921988401: [3, 4, 3], avg=4.37
Injecting a

# Advertisement Classification and Evaluation

In the final section, we evaluate whether our generated responses correctly integrate advertisements. A zero-shot classifier (`facebook/bart-large-mnli`) is used to detect the presence of advertisement content in each response.

**Evaluation Details:**
- **`ad_classifier` Function:**  
  Classifies text as either "advertisement" or "non-advertisement" using candidate labels.
- **`evaluate_ad_integration` Function:**  
  Compares the classifier predictions against the ground truth labels (determined by whether the advertisement field is empty or not).  
  It calculates precision and recall metrics to assess the accuracy of advertisement integration.
- **Loading and Evaluating Responses:**  
  The responses saved to the JSONL file are loaded back, and the evaluation metrics are printed.

This evaluation helps ensure that the response generation logic meets the desired criteria for ad integration.


In [15]:
# ------------------------------
# Load data and run evaluation
# ------------------------------
output_file = "/kaggle/working/subtask1_responses.jsonl"
responses_output = []
with open(output_file, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            responses_output.append(json.loads(line))

generated_responses = [entry["response"] for entry in responses_output]
ground_truth_labels = [1 if entry["advertisement"].strip() else 0 for entry in responses_output]

# Evaluate with threshold
precision, recall, predictions = evaluate_ad_integration(
    generated_responses,
    ground_truth_labels,
    threshold=0.95  # Lower this threshold if ads are too evasive
)

print("Ad Classifier Predictions:", predictions)
print("Ground Truth Labels:     ", ground_truth_labels)
print(f"Precision (Ads): {precision:.3f}")
print(f"Recall (Ads):    {recall:.3f}")

Ad Classifier Predictions: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Ground Truth Labels:      [0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 

In [16]:
import json

file_path = "/kaggle/working/subtask1_responses.jsonl"

# Read and print the first few entries
with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 100:
            break
        entry = json.loads(line)
        print(f"Response ID: {entry['id']}")
        print("Topic:", entry["topic"])
        print("Response:", entry["response"])
        print("Advertisement:", entry["advertisement"])
        print("References:", entry["references"])
        print("-" * 60)


Response ID: 693413000-0
Topic: money bank
Response: Inside money refers to the form of money that exists in bank accounts as bank deposits and is created as a result of loans. This form of money is controlled almost entirely by private banks and is created through the lending process. Online-only banks tend to offer higher average rates on savings accounts and high-yield checking compared to brick-and-mortar banks, so it's no surprise that their money market accounts offer better APYs as well. Major online-only banks include ableBanking, Ally Bank, Synchrony Bank, Discover Bank, Axos Bank, Capital One, Radius Bank, and TIAA Bank. The rates for money market accounts at these banks range from 0.30% to 0.60% depending on the balance.
Advertisement: 
References: ['msmarco_v2.1_doc_49_1656341109#3_3478204805', 'msmarco_v2.1_doc_49_1656341109#4_3478205983', 'msmarco_v2.1_doc_56_1174342110#6_2346964429']
------------------------------------------------------------
Response ID: 693413000-1
To

In [17]:
from collections import Counter

with open("/kaggle/working/subtask1_responses.jsonl", "r", encoding="utf-8") as f:
    ads = [json.loads(line)["advertisement"] for line in f if line.strip()]

ad_counts = Counter([ad for ad in ads if ad.strip() != ""])
print("Ad mentions summary:")
for ad, count in ad_counts.most_common():
    print(f"{ad}: {count}")


Ad mentions summary:
First Citizens Bank: 6
PNC Bank: 5
Chase: 5
Monzo: 5
TD Beyond Checking and Savings Accounts: 4
USAA Classic Checking: 4
Mint: 4
Revolut: 4
N26: 4
HSBC: 3
PNC Virtual Wallet: 3
Citizens Bank: 3
Bank of America Business Advantage Relationship Rewards: 3
Citibank Access Checking: 3
TD Bank: 3
HSBC Premier Checking: 3
Barclays: 2
Citi: 2
Bank of the West: 2
Saxo Bank: 2
Bank of America: 2
ING: 2
Merrill: 2
Ally Bank: 2
Alliant Credit Union: 2
Union Bank Home Loans: 2
Bank of America Advantage Plus: 2
Bank of America Customized Cash Rewards: 2
Morgan Stanley: 1
SunTrust Bank: 1
Barclays AAdvantage Aviator Red: 1
Goldman Sachs: 1
Marcus by Goldman Sachs: 1
Webster Bank Value Checking: 1
Wells Fargo Loans: 1
Fifth Third Preferred Credit Card: 1
Fifth Third Bank: 1
Visa: 1
Discover Cash Back Credit Card: 1
Chase Freedom Flex: 1
BMO Harris Bank Premium Rewards Mastercard: 1
Citi Bank Premier Savings: 1
American Express: 1
Capital One: 1
Banco Santander: 1
Santander Simply 